In [9]:
import wrf_management.run_utilities
import wrf_management.utilities as ut
import importlib
importlib.reload(ut);
import wrf_management.project_global_constants as gc
importlib.reload(gc)
import wrf_management.geogrid as geo
import wrf_management.ungrib as un
importlib.reload(un)
import os
import sqlite3 as sq
import pandas as pd
import wrf_management.base_namelists.base_namelists as bn
importlib.reload(bn);
import f90nml
import subprocess as su

In [10]:
print(gc.RUN_NAME)
job = 'ungrib_surf'
file_types = ['surf_0', 'surf_1']

real = True
hours = 24

LIST_S_LINKS = [
    'ungrib.exe',
    'link_grib.csh',
    'ungrib'
]

LIST_H_LINKS = [
    'Vtable',
    'env_WRFv4.bash'
]

run_2019_02_20


In [11]:
#con = sq.connect(gc.PATH_DB)
gc.PATH_DB

'/homeappl/home/aliagadi/saltena_2018/wrf_management/wrf_management/db_folder/wrf_man.sqlite'

In [12]:
importlib.reload(un)
run_row = wrf_management.run_utilities.get_run_row()
print(run_row)

index                             1
run_name             run_2019_02_20
geogrid_path                geogrid
config_path          run_2019_02_20
data_path       runs/run_2019_02_20
parent_run                     None
comments                       None
Name: 0, dtype: object


In [13]:
job_row = wrf_management.run_utilities.get_next_row(job=job, i_max=10)
print(job_row)

date              2018-01-16 00:00:00
ungrib_surf                         5
ungrib_press                      200
ungrib_avgtsfc                      0
metgrid                             0
real                                0
Name: 0, dtype: object


In [14]:
if real:
    wrf_management.run_utilities.update_run_table(val=job_row[job] + 1,
                                                  job=job,
                                                  date=job_row['date']
                                                  )

job_path = wrf_management.run_utilities.getmk_job_path(run_row, job_row, job)
print(job_path)

wrf_management.run_utilities.rm_if_path_exists(
    os.path.join(job_path, 'ungrib.log')
)

/wrk/aliagadi/DONOTREMOVE/wrf_management_data/runs/run_2019_02_20/2018_01_16/ungrib_surf


In [15]:
conf_path = wrf_management.run_utilities.get_conf_path(run_row)
print(conf_path)

type_rows = pd.DataFrame([un.get_type_row(ft, job_row) for ft in file_types])
print(type_rows)

name_list = un.skim_namelist_copy(
    conf_path, job_path, date=job_row.date, prefix=job, hours=hours
)
print(name_list)

/homeappl/home/aliagadi/saltena_2018/wrf_management/wrf_management/config_dir/run_2019_02_20
                  date   i  downloaded                          name  \
0  2018-01-16 00:00:00  46           1    cdas1.20180116.splgrbf.tar   
0  2018-01-16 00:00:00  46           1  cdas1.20180116.sfluxgrbf.tar   

   untarred    type  
0         0  surf_0  
0         0  surf_1  
OrderedDict([('share', Namelist([('wrf_core', 'ARW'), ('max_dom', 4), ('start_date', ['2018-01-16_00:00:00', '2018-01-16_00:00:00', '2018-01-16_00:00:00', '2018-01-16_00:00:00']), ('end_date', ['2018-01-17_00:00:00', '2018-01-17_00:00:00', '2018-01-17_00:00:00', '2018-01-17_00:00:00']), ('interval_seconds', 21600), ('io_form_geogrid', 2)])), ('ungrib', Namelist([('out_format', 'WPS'), ('prefix', 'ungrib_surf')]))])


In [16]:
if gc.ID=='taito_login':
    wrf_management.run_utilities.copy_hard_links(conf_path, job_path, LIST_H_LINKS)
    wrf_management.run_utilities.copy_soft_links(gc.PATH_WPS, job_path, LIST_S_LINKS)
    importlib.reload(un)
    un.untar_the_files(type_rows, job_path, job_row=job_row)
    
    # in case we need to download the day before
    if job == 'ungrib_surf':
        pre_row = wrf_management.run_utilities.get_prev_row(job=job, job_row=job_row)
        trs = pd.DataFrame([un.get_type_row(ft, pre_row) for ft in file_types])
        un.untar_the_files_prev(trs, job_path, job_row=pre_row)

Vtable
env_WRFv4.bash
unilinking
ungrib.exe
unilinking
link_grib.csh
unilinking
ungrib
/wrk/aliagadi/DONOTREMOVE/wrf_management_data/surf_0_tar/cdas1.20180116.splgrbf.tar
/wrk/aliagadi/DONOTREMOVE/wrf_management_data/surf_1_tar/cdas1.20180116.sfluxgrbf.tar
/wrk/aliagadi/DONOTREMOVE/wrf_management_data/surf_0_tar/cdas1.20180115.splgrbf.tar
cdas1.t18z.splgrbf06.grib2
/wrk/aliagadi/DONOTREMOVE/wrf_management_data/surf_1_tar/cdas1.20180115.sfluxgrbf.tar
cdas1.t18z.sfluxgrbf01.grib2
cdas1.t18z.sfluxgrbf02.grib2
cdas1.t18z.sfluxgrbf03.grib2
cdas1.t18z.sfluxgrbf04.grib2
cdas1.t18z.sfluxgrbf05.grib2
cdas1.t18z.sfluxgrbf06.grib2
cdas1.t18z.sfluxgrbf07.grib2
cdas1.t18z.sfluxgrbf08.grib2
cdas1.t18z.sfluxgrbf09.grib2


In [17]:
run_script = \
    """#!/bin/bash
        
cd {job_path}
./link_grib.csh ./untar/*/*
source ./env_WRFv4.bash 
srun -t20 -p serial --mem 1000 -J's{date}' ./ungrib.exe
exit $?
    
    """.format(job_path=job_path, date=job_row.date)
print(run_script)
bs_path = os.path.join(job_path, 'run_me.sh')
bs_file = open(bs_path, 'w')
bs_file.write(run_script)
bs_file.close()

#!/bin/bash
        
cd /wrk/aliagadi/DONOTREMOVE/wrf_management_data/runs/run_2019_02_20/2018_01_16/ungrib_surf
./link_grib.csh ./untar/*/*
source ./env_WRFv4.bash 
srun -t20 -p test --mem 1000 ./ungrib.exe
exit $?
    
    


In [18]:
if gc.ID == 'taito_login':
    res = su.run(['/bin/bash', bs_path], stdout=su.PIPE, stderr=su.PIPE)

print(res.stdout)
print(res.stderr)
if gc.ID == 'taito_login' and res.returncode == 0:
    wrf_management.run_utilities.update_run_table(val=100,
                                                  job=job,
                                                  date=job_row['date']
                                                  )

b' *** Starting program ungrib.exe ***\nStart_date =  2018-01-16_00:00:00 ,      End_date = 2018-01-17_00:00:00\noutput format is WPS\nPath to intermediate files is ./\n\n###############################################################################\n\nInventory for date = 2018-01-15 19:00:00\n\nPRES   TT       UU       VV       RH       HGT      PMSL     SPECHUMD PSFC     SM000010 SM010040 SM040100 SM100200 ST000010 ST010040 ST040100 ST100200 SEAICE   LANDSEA  SOILHGT  SKINTEMP SNOW    \n-------------------------------------------------------------------------------\n2001.0  X        X        X        O        O        O        X        X        X        X        X        X        X        X        X        X        X        X        X        X        X      \n-------------------------------------------------------------------------------\n\n###############################################################################\n\nInventory for date = 2018-01-15 20:00:00\n\nPRES   TT       U